In [1]:
import json
import pandas as pd
import numpy as np
from FlagEmbedding import BGEM3FlagModel
from tqdm.auto import tqdm

/home/student/aboba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
descriptions = json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json'))
descriptions = {' '.join(label.split('_')): descriptions[label] for label in list(descriptions)}
labels = list(descriptions)

In [4]:
model = BGEM3FlagModel('BAAI/bge-m3', use_bfp16=True) 

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 150693.56it/s]


In [5]:
n = len(descriptions)
pairs = []
nearest_labels = {}

for i in tqdm(range(n)):
    embeddings_1 = model.encode(descriptions[labels[i]])['dense_vecs']
    cur_nearest = []
    max_similarity = 0
    for j in range(n):
        if i != j:
            embeddings_2 = model.encode(descriptions[labels[j]])['dense_vecs']
            similarity = embeddings_1 @ embeddings_2.T
            if similarity >= max_similarity:
                cur_nearest.append(labels[j])
                max_similarity = similarity
                if len(cur_nearest) > 3:
                    cur_nearest.pop(0)
                
    nearest_labels[labels[i]] = cur_nearest

100%|██████████| 150/150 [16:25<00:00,  6.57s/it]


In [ ]:
nearest_labels

In [7]:
with open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json', "w") as f:
    json.dump(descriptions, f, indent=4)
with open('/home/student/katya/tbank-ml-camp-sirius/data/nearest_labels.json', "w") as f:
    json.dump(nearest_labels, f, indent=4)

In [3]:
descriptions = json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json'))
descriptions = {'_'.join(label.split()): descriptions[label]  for label in list(descriptions)}

In [9]:
nearest_labels = json.load(open('/home/student/katya/tbank-ml-camp-sirius/data/nearest_labels.json'))
new_nearest_labels = {}
for label in list(nearest_labels):
    nearest = nearest_labels[label]
    new_nearest_labels['_'.join(label.split())] = ['_'.join(l.split()) for l in nearest]

In [6]:
new_nearest_labels = {}
for label in list(nearest_labels):
    nearest = nearest_labels[label]
    new_nearest_labels['_'.join(label.split())] = ['_'.join(l.split()) for l in nearest]

In [10]:
with open('/home/student/katya/tbank-ml-camp-sirius/data/labels_with_description.json', "w") as f:
    json.dump(descriptions, f, indent=4)
with open('/home/student/katya/tbank-ml-camp-sirius/data/nearest_labels.json', "w") as f:
    json.dump(new_nearest_labels, f, indent=4)